# Subset Sum and Knapsack Problem

Lucerne University of Applied Sciences and Arts - School of Information Technology

In the knapsack problem, you need to pack a set of items, with given values and sizes (such as weights or volumes), into a container with a maximum capacity. If the total size of the items exceeds the capacity, you can't pack them all. In that case, the problem is to choose a subset of the items of maximum total value that will fit in the container. Subset sum is a variant of the knapsack problem where items have constant values. If the binary variant of the knapsack problem each item can be selected only once.

Text, code snippets and inspiration from [here](https://developers.google.com/optimization/bin/knapsack); more examples can be found [here](https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html)

@author: Tobias Mérinat and Marc Pouly

Imports

In [1]:
from ortools.sat.python import cp_model
from ortools.algorithms import pywrapknapsack_solver

### Select an example

In [2]:
example = 'MEDIUM'

if example == 'XKCD':
    values   = [1, 1, 1, 1, 1, 1]
    names    = ['Fruit', 'Fries', 'Salad', 'Wings', 'Sticks', 'Plate']
    weights  = [215, 275, 335, 355, 420, 580]
    capacity = 1505

elif example == 'TINY':
    values   = [15, 10, 7]
    names    = ["Whiskey", "Perfume", "Cigarettes"]
    weights  = [4, 3, 2]
    capacity = 9
    
elif example == 'SMALL':
    values   = [12, 8, 2, 15]
    names    = ["Whiskey", "Perfume", "Corned Beef", "Riffle"]
    weights  = [4, 3, 2, 6]
    capacity = 29
    
elif example == 'MEDIUM':
    values   = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
    names    = [f"Item {i}" for i, _ in enumerate(values)]
    weights  = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
    capacity = 750
    # Optimal profit for 0/1 variant: 1458
    # Optimal profit for unrestricted variant: 1488
    
else:
    print('Invalid data set identifies.')

Choose whether to solve the 01 Knapsack problem (pick at most one of each items)

In [3]:
binary_variant = True

## Model 1: My own model

Create constraint solver

In [4]:
model = cp_model.CpModel()

Configure variables (the number of times an item is chosen)

In [5]:
if binary_variant:
    choices = [model.NewBoolVar(str(i)) for i in weights]
else:
    choices = [model.NewIntVar(0, capacity, str(i)) for i in weights]
    

Total weight must not exceed capacity

In [6]:
model.Add(cp_model.LinearExpr.ScalProd(choices, weights) <= capacity)

Price objective (solver.Difference is not available in the Python wrapper)

In [7]:
model.Maximize(cp_model.LinearExpr.ScalProd(choices, values))

Configure solver

In [8]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print(f"Optimal objective value: {int(solver.ObjectiveValue())}")
    for i, choice in enumerate(choices):
        print(f" - Item {i:>2} with value {values[i]} and weight {weights[i]:>3} "
              f"has been chosen {solver.Value(choice)} times")
    
elif status == cp_model.INFEASIBLE:
    print("The model is over-constrained.")

Optimal objective value: 1458
 - Item  0 with value 135 and weight  70 has been chosen 1 times
 - Item  1 with value 139 and weight  73 has been chosen 0 times
 - Item  2 with value 149 and weight  77 has been chosen 1 times
 - Item  3 with value 150 and weight  80 has been chosen 0 times
 - Item  4 with value 156 and weight  82 has been chosen 1 times
 - Item  5 with value 163 and weight  87 has been chosen 0 times
 - Item  6 with value 173 and weight  90 has been chosen 1 times
 - Item  7 with value 184 and weight  94 has been chosen 1 times
 - Item  8 with value 192 and weight  98 has been chosen 1 times
 - Item  9 with value 201 and weight 106 has been chosen 0 times
 - Item 10 with value 210 and weight 110 has been chosen 0 times
 - Item 11 with value 214 and weight 113 has been chosen 0 times
 - Item 12 with value 221 and weight 115 has been chosen 0 times
 - Item 13 with value 229 and weight 118 has been chosen 1 times
 - Item 14 with value 240 and weight 120 has been chosen 1 t

## Model 2: Built-in specialized Knapsack model (only 0-1 variant)

In [9]:
if not binary_variant:
    print('Built-in Knapsack solver can only solve 0-1 variant of Knapsack. Code below will crash.')

In [10]:
solver = pywrapknapsack_solver.KnapsackSolver(pywrapknapsack_solver.KnapsackSolver.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, '')

In [11]:
# Built-in model even accepts multiple container with individual capacities as well as contained specific weights
# It therefore expects a weight matrix and a list of container capacities
solver.Init(values, [weights], [capacity])

# Calling the solver directly gives the optimal value
best = solver.Solve()

# Printing solutions: solver.BestSolutionContains(c) plays the role of our choices array in model 1
print(f"Optimal objective value: {best}")

for i, _ in enumerate(values):
    print(f" - Item {i:>2} with value {values[i]} and weight {weights[i]:>3} "
          f"has been chosen {int(solver.BestSolutionContains(i))} times")

Optimal objective value: 1458
 - Item  0 with value 135 and weight  70 has been chosen 1 times
 - Item  1 with value 139 and weight  73 has been chosen 0 times
 - Item  2 with value 149 and weight  77 has been chosen 1 times
 - Item  3 with value 150 and weight  80 has been chosen 0 times
 - Item  4 with value 156 and weight  82 has been chosen 1 times
 - Item  5 with value 163 and weight  87 has been chosen 0 times
 - Item  6 with value 173 and weight  90 has been chosen 1 times
 - Item  7 with value 184 and weight  94 has been chosen 1 times
 - Item  8 with value 192 and weight  98 has been chosen 1 times
 - Item  9 with value 201 and weight 106 has been chosen 0 times
 - Item 10 with value 210 and weight 110 has been chosen 0 times
 - Item 11 with value 214 and weight 113 has been chosen 0 times
 - Item 12 with value 221 and weight 115 has been chosen 0 times
 - Item 13 with value 229 and weight 118 has been chosen 1 times
 - Item 14 with value 240 and weight 120 has been chosen 1 t